# Design and Analyze a beam with holes

The following example demonstrates how you can add penetrations to a beam using various primitives and finally
export it to an IFC file

In [ ]:
import numpy as np

import ada
from ada.fem.meshing import GmshOptions
from ada.materials.metals import CarbonSteel, DnvGl16Mat

**The basic beam setup**

The following code creates the following Assembly (and IFC) hierarchy

    MyAssembly (IfSite)
        MyPart (IfcBuildingStorey)
            MyBeam (IfcBeam)

In [ ]:
bm = ada.Beam(
    "MyBeam",
    (0, 0, 0),
    (1.5, 0, 0),
    "IPE400",
    ada.Material("S420", CarbonSteel("S420", plasticity_model=DnvGl16Mat(0.1, "S420"))),
)
a = ada.Assembly("MyAssembly") / [ada.Part("MyPart") / bm]
a.show()

**Polygon Extrusions**

In [ ]:
h = 0.2
r = 0.02

normal = [0, 1, 0]
xdir = [-1, 0, 0]


origin = np.array([0.2, -0.1, -0.1])
points = [(0, 0), (0.1, 0), (0.05, 0.1)]

poly1 = bm.add_boolean(ada.PrimExtrude("Poly1", points, h, normal, origin, xdir))
bm.show()

In [ ]:
origin += np.array([0.2, 0, 0])
points = [(0, 0, r), (0.1, 0, r), (0.05, 0.1, r)]

poly2 = bm.add_boolean(ada.PrimExtrude("Poly2", points, h, normal, origin, xdir))
bm.show()

In [ ]:
origin += np.array([0.2, 0, 0])
points = [(0, 0, r), (0.1, 0, r), (0.1, 0.2, r), (0.0, 0.2, r)]

poly3 = bm.add_boolean(ada.PrimExtrude("Poly3", points, h, normal, origin, xdir))
bm.show()

In [ ]:
# Cylinder Extrude
x = origin[0] + 0.2

cyl = bm.add_boolean(ada.PrimCyl("cylinder", (x, -0.1, 0), (x, 0.1, 0), 0.1))
bm.show()

In [ ]:
# Box Extrude
x += 0.2

box = bm.add_boolean(ada.PrimBox("box", (x, -0.1, -0.1), (x + 0.2, 0.1, 0.1)))
bm.show()

In [ ]:
# Export IFC to the Home folder
a.to_ifc("../output/MyBeamWithHoles.ifc")

In [ ]:
# Create a FEM analysis of the beam as a cantilever subjected to gravity loads
p = a.get_part("MyPart")
p.fem = bm.to_fem_obj(0.1, "shell", options=GmshOptions(Mesh_MeshSizeFromCurvature=True))

a.show()

In [ ]:
# Create a Static Analysis Step with a Gravity load (multiplied with 800 to get deformation)
step = a.fem.add_step(ada.fem.StepImplicitStatic("gravity", nl_geom=True, init_incr=100.0, total_time=100.0))
step.add_load(ada.fem.LoadGravity("grav", -9.81 * 800))

nodes = bm.bbox().sides.back(return_fem_nodes=True)
a.fem.add_bc(ada.fem.Bc("Fixed", ada.fem.FemSet("bc_nodes", nodes), [1, 2, 3]))

In [ ]:
res = a.to_fem("MyCantilever_code_aster", "code_aster", execute=True, overwrite=True)

In [ ]:
res

In [ ]:
# Export IFC to the Home folder
a.to_ifc("../output/MyBeamWithHoles.ifc")

**Add the primitives used for cutting as solid shapes to your model**

In [ ]:
a / (ada.Part("Shapes") / [x.primitive for x in [poly1, poly2, poly3, cyl, box]])
a

In [ ]:
# Export IFC to the Home folder
a.to_ifc("../output/MyBeamWithHoles_and_Primitives.ifc")